In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 这一行注释掉就是使用gpu，不注释就是使用cpu

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

d:\programfile\python3.7.4\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\programfile\python3.7.4\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\programfile\python3.7.4\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\programfile\python3.7.4\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [3]:
class gNN:
    def __init__(self, x, layers, activation, lr, weight_g):
        self.x = x
        self.lr = lr
        self.activation = activation
        self.weight_g = weight_g
        self.loss_log = []
        self.loss_res_log = []
        self.loss_deriv_log = []

        self.weights, self.biases = self.initilize_NN(layers)
        self.sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])

        self.surrogate_u = self.surrogate(self.x_tf)
        self.residual    = self.residual(self.surrogate_u, self.x_tf)
        self.loss_res    = tf.reduce_mean(tf.square(self.residual))
        self.loss_deri   = tf.reduce_mean(tf.square(self.resi_deriv(self.residual, self.x_tf)))
        self.loss        = self.loss_res + self.weight_g * self.loss_deri

        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initilize_NN(self, layers):
        weights = []
        biases  = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

    def neural_net(self, X, weights, biases, activation):
        num_layers = len(weights) + 1
        H = X
        for l in range(0, num_layers - 2):
            W = weights[l]
            b = biases[l]
            H = activation(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    def net_u(self, x):
        u = self.neural_net(x, self.weights, self.biases, self.activation)
        return u

    def surrogate(self, x):
        return x*(np.pi - x)*self.net_u(x) + x

    def residual(self, u, x):
        u_x  = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        rhs  = 0
        for i in range(1, 5):
            rhs += i*tf.sin(i*x)
        rhs += 8*tf.sin(8*x)
        return u_xx + rhs

    def resi_deriv(self, res, x):
        res_x = tf.gradients(res, x)[0]
        return res_x

    def callback(self, loss, res, deriv):
        print('Loss:%f,res:%f,deriv:%f'%(loss, res, deriv))

    def train(self, max_iter=40000):
        loss_value = np.inf
        for iter in range(max_iter):
            tf_dict = {
                self.x_tf:self.x
            }
            _, loss_value, lo_res, lo_deriv = self.sess.run([self.optimizer,
                self.loss, self.loss_res, self.loss_deri], tf_dict)
            if iter % 1000 == 0:
                print("第%d次 %f,res %f,deriv %f"%(iter, loss_value, lo_res, lo_deriv))
        print("第%d次的损失为%f"%(max_iter, loss_value))

    def predict(self, X_star):
        u_star = self.sess.run(self.surrogate_u, {self.x_tf: X_star})
        return u_star

In [10]:
layers = [1, 20, 20, 20, 20, 1]
activation = tf.tanh
lr = 1e-3
weight_g = 1
iterations = 20000

In [5]:
def train_model(train_num):
    x = np.linspace(0, np.pi, train_num)[:, None]
    start_time = time.time()
    gNN_tanh = gNN(x, layers, activation, lr, weight_g)
    print("Start training! train_num:%d"%(train_num))
    gNN_tanh.train(iterations)
    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))
    return gNN_tanh

In [6]:
def compute_error(model, test_num):
    x_test   = np.linspace(0, np.pi, test_num)[:, None]
    u_test   = x_test
    for i in range(1, 5):
        u_test += np.sin(i*x_test) / i
    u_test  +=  np.sin(8*x_test)/8
    u_pred   = model.predict(x_test)
    L2_norm  = np.linalg.norm(u_test-u_pred,2)/np.linalg.norm(u_test,2)
    return L2_norm

In [11]:
train_nums = np.arange(10, 21, 1)
test_num   = 1000
errors     = []
for train_num in train_nums:
    model = train_model(train_num)
    error = compute_error(model, test_num)
    print("train_num:%d, error:%f"%(train_num, error))
    errors.append(error)

Start training! train_num:10
第0次 2762.393555,res 41.445503,deriv 2720.947998
第1000次 1193.260864,res 17.184135,deriv 1176.076782
第2000次 968.923706,res 14.549777,deriv 954.373901
第3000次 4.423996,res 0.443858,deriv 3.980138
第4000次 0.000308,res 0.000114,deriv 0.000194
第5000次 0.045208,res 0.026935,deriv 0.018273
第6000次 0.008901,res 0.006367,deriv 0.002534
第7000次 0.023326,res 0.001310,deriv 0.022016
第8000次 0.038697,res 0.006928,deriv 0.031769
第9000次 0.498456,res 0.005853,deriv 0.492603
第10000次 0.043448,res 0.003377,deriv 0.040071
第11000次 1.595584,res 0.009813,deriv 1.585771
第12000次 0.419789,res 0.018927,deriv 0.400863
第13000次 0.150977,res 0.006751,deriv 0.144226
第14000次 0.081023,res 0.005293,deriv 0.075730
第15000次 0.250766,res 0.028284,deriv 0.222483
第16000次 0.021832,res 0.001568,deriv 0.020264
第17000次 0.223306,res 0.018731,deriv 0.204574
第18000次 0.018318,res 0.001904,deriv 0.016415
第19000次 0.013748,res 0.000821,deriv 0.012927
第20000次的损失为0.050226
Training time: 111.7675
train_num:10, error:0

In [12]:
errors_gPINN = np.array(errors)
np.save('errors_gPINN.npy', errors_gPINN)

In [9]:
# from tensorflow.python.client import device_lib as _device_lib
# local_device_protos = _device_lib.list_local_devices()
# devices = [x.name for x in local_device_protos]
# for d in devices:
# 	print(d)
